<a href="https://colab.research.google.com/github/mhdykz/processing-techniques-to-denoise-images-corrupted-by-salt-and-pepper-noise/blob/main/classic_image_processing_pro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


In [2]:
def remove_strong_noise(image):
    noisy_image = image.copy()
    noisy_image[noisy_image == 0] = 1
    noisy_image[noisy_image == 255] = 254
    return noisy_image